In [1]:
from geopy.geocoders import Nominatim
import geopy
from math import *
import networkx as nx
import pandas as pd
import random
import openpyxl

In [2]:
def distance_between(coords1,coords2):
    radius_of_earth = 6373
    lat1 = radians(coords1[0])
    lon1 = radians(coords1[1])
    lat2 = radians(coords2[0])
    lon2 = radians(coords2[1])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return round(radius_of_earth*c,2)

In [3]:
cartels = pd.read_csv('BACRIM2020_Nodes.csv')
alliances = pd.read_csv('BACRIM2020_Alliances.csv')
rivals = pd.read_csv('BACRIM2020_Rivals.csv')
members = pd.read_excel('BACRIM2020_individuals.xlsx')
states = pd.read_csv("states.csv")

In [4]:
coordinates = dict()
for i in list(states['place']):
    lat = float(states.loc[states['place']==i]["latitude"])
    long = float(states.loc[states['place']==i]["longitude"])
    coordinates[i] = [lat,long]

C:\Users\Pranav Sridhar\AppData\Local\Temp\ipykernel_24932\208044534.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lat = float(states.loc[states['place']==i]["latitude"])
C:\Users\Pranav Sridhar\AppData\Local\Temp\ipykernel_24932\208044534.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  long = float(states.loc[states['place']==i]["longitude"])


In [5]:
graph = nx.DiGraph()

In [6]:
for i in list(states['place']):
    graph.add_node(i)

In [7]:
for i in list(states['place']):
    for j in list(states['place']):
        if i!=j and distance_between(coordinates[i],coordinates[j])<510:
            graph.add_edge(i,j,label = "IS_NEIGHBOUR")
            graph.add_edge(j,i,label = "IS_NEIGHBOUR")

In [8]:
for i in range(len(cartels['Group'])):
    cartel = cartels['Group'][i]
    graph.add_node(cartel)
    graph.add_edge(cartel,cartels['State'][i],label = "IN_REGION")

In [9]:
for i in range(len(alliances['Group'])):
    graph.add_edge(alliances['Group'][i],alliances['RGroup'][i],label = "IS_ALLY")

for i in range(len(rivals['Group'])):
    graph.add_edge(rivals['Group'][i],rivals['RGroup'][i],label = "IS_RIVAL")

In [10]:
true_members = members.dropna(subset=['person'])
true_members = true_members.reset_index()
true_members = true_members.drop(columns=['index'])

In [11]:
for i in range(len(true_members['person'])):
    cart_members = true_members['person'][i].split(sep='; ')
    if true_members['group'][i] in list(graph.nodes()):
        for j in cart_members:
            name = j.split(', ')[0]
            graph.add_node(name)
            graph.nodes[name]['type'] = 'person'
            graph.add_edge(name,true_members['state'][i],label = "WAS_SPOTTED_IN")
            graph.add_edge(name,true_members['group'][i],label = "IS_PART_OF")

In [12]:
import matplotlib.pyplot as plt

In [16]:
from matplotlib import pylab
import networkx as nx

def save_graph(graph,file_name):
    #initialze Figure
    plt.figure(num=None, figsize=(20, 20), dpi=80)
    plt.axis('off')
    fig = plt.figure(1)
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph,pos)
    nx.draw_networkx_edges(graph,pos)
    nx.draw_networkx_labels(graph,pos)

    cut = 1.00
    xmax = cut * max(xx for xx, yy in pos.values())
    ymax = cut * max(yy for xx, yy in pos.values())
    plt.xlim(0, xmax)
    plt.ylim(0, ymax)

    plt.savefig(file_name,bbox_inches="tight")
    pylab.close()
    del fig

#Assuming that the graph g has nodes and edges entered
save_graph(graph,"my_graph.pdf")

In [19]:
nx.write_graphml(graph, 'cartels.graphml')